## Import Libraries

In [1]:
import os
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, start, listen  
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun



## Load the  LLM


In [2]:
from crewai import LLM

llm = LLM(
    model="groq/qwen/qwen3-32b",               # Updated model name
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1"
)


## Load Tools

In [3]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Agent Definitions



In [4]:
loader_agent = Agent(
    role="Loader Agent",
    goal="Search and retrieve electric consumption data for Indian states (2018-2023).",
    backstory="A research assistant who finds credible data sources from the web.",
    tools=[search_tool1],
    llm=llm,
    verbose=True
)

In [5]:
from langchain_experimental.tools import PythonREPLTool


@tool("analyzer")
def search_tool2(query: str) -> str:
    """Executes Python code for data analysis or visualization."""
    return PythonREPLTool().run(query)


In [6]:
analyzer_agent = Agent(
    role="Analyzer Agent",
    goal="Analyze the CSV to find top 3 states with highest average consumption.",
    backstory="A data analyst who uses pandas to extract patterns.",
    tools=[search_tool2],
    llm=llm,
    verbose=True
)
#This one loads CSV and analyzes with pandas.

In [7]:
visualizer_agent = Agent(
    role="Visualizer Agent",
    goal="Create bar charts to show electricity consumption per year by state.",
    backstory="A matplotlib expert for Indian state electricity data.",
    tools=[search_tool2],
    llm=llm,
    verbose=True
)
#Makes visual plots of electricity data.

In [8]:
summarizer_agent = Agent(
    role="Summarizer Agent",
    goal="Write a 2-3 sentence summary about which states consumed the most electricity and why.",
    backstory="A professional executive summary writer.",
    tools=[],
    llm=llm,
    verbose=True
)
#Pure LLM-based summary writer.



## Task Definitions

In [9]:
load_task = Task(
    description="Search for electricity consumption datasets for Indian states (2018–2023).",
    agent=loader_agent,
    expected_output="A list of 3 URLs to government or trusted sources",
    input_type="input"
    )


analyze_task = Task(
    description="Load a CSV file, calculate average consumption, and print top 3 states.",
    agent=analyzer_agent,
    expected_output="A list of the top 3 Indian states with the highest average electricity consumption."
)

visualize_task = Task(
    description="Use matplotlib/seaborn to plot electricity consumption for each state.",
    agent=visualizer_agent,
    expected_output="A bar chart or line graph showing electricity consumption for each Indian state."
)

summarize_task = Task(
    description="Write a summary of insights from the data and visualizations in 2-3 sentences.",
    agent=summarizer_agent,
    expected_output="A concise textual summary of the most important insights from the analysis and visualizations."
)


combine_task = Task(
    description="Combine outputs from analyzer, visualizer, and summarizer into a single report.",
    agent=summarizer_agent,
    # optional: context=[analyze_task, visualize_task, summarize_task],  # not required with sequential
    expected_output="A single, well-structured final report summarizing the data, chart insights, and top 3 states."
)



## Flow Definition

In [10]:
from crewai import Crew, Process

#  Step 1: Run the loader agent first (sequential)
loader_crew = Crew(
    agents=[loader_agent],
    tasks=[load_task],
    process=Process.sequential
)

#  Get the context (output from loader_agent)
context = loader_crew.kickoff(inputs={"input": "Search for electricity consumption datasets for Indian states (2018–2023). Focus on government or trusted sources. Return top 3 links."})



#  Step 2: Set the next tasks to run in parallel
analyze_task.async_execution = True
visualize_task.async_execution = True
summarize_task.async_execution = True
combine_task.async_execution = False

#  Step 3: Run the 3 agents in parallel using shared context
main_crew = Crew(
    agents=[analyzer_agent, visualizer_agent, summarizer_agent],
    tasks=[analyze_task, visualize_task, summarize_task,combine_task],
    process=Process.sequential
)




# Convert CrewOutput to plain string
text = context.raw

# Now run main_crew with string input
final_result = main_crew.kickoff(inputs={"input": text})
print(final_result)




╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loader Agent                                                                                            │
│                                                                                                                 │
│  Task: Search for electricity consumption datasets for Indian states (2018–2023).                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loader Agent                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  [                                                                                                              │
│    {                                                                                                            │
│      "url": "https://www.powermin.gov.in/sites/default/files/publication/annual_report_2022-23.pdf"             │
│    },                                                                                                           │
│    {                                                                                                            │
│      "url":                                                                                                     │
│  "http://en.cea.nic.in/index.php?option=com_content&view=article&id=98:state-wise-energy-statistics&catid=39:e  │
│  nergy-statistics"                                                                                              │
│    },                                                                                                           │
│    {                                                                                                            │
│      "url": "https://indiaenergyforum.in/india-energy-statistics"                                               │
│    }                                                                                                            │
│  ]                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analyzer Agent                                                                                          │
│                                                                                                                 │
│  Task: Load a CSV file, calculate average consumption, and print top 3 states.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Task: Write a summary of insights from the data and visualizations in 2-3 sentences.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Visualizer Agent                                                                                        │
│                                                                                                                 │
│  Task: Use matplotlib/seaborn to plot electricity consumption for each state.                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Visualizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```python                                                                                                      │
│  import matplotlib.pyplot as plt                                                                                │
│  import seaborn as sns                                                                                          │
│  import pandas as pd                                                                                            │
│                                                                                                                 │
│  # Sample data for electricity consumption (in TWh) by Indian states for 2020-2021                              │
│  data = {                                                                                                       │
│      'State': ['Maharashtra', 'Tamil Nadu', 'UP', 'Karnataka', 'Kerala'],                                       │
│      '2020': [120, 90, 80, 70, 40],                                                                             │
│      '2021': [130, 95, 85, 75, 42]                                                                              │
│  }                                                                                                              │
│                                                                                                                 │
│  df = pd.DataFrame(data).set_index('State')                                                                     │
│                                                                                                                 │
│  # Convert to long format for plotting                                                                          │
│  df_melted = df.reset_index().melt(id_vars='State', var_name='Year', value_name='Consumption (TWh)')            │
│                                                                                                                 │
│  # Plotting                                                                                                     │
│  plt.figure(figsize=(12, 6))                                                                                    │
│  sns.barplot(x='State', y='Consumption (TWh)', hue='Year', data=df_melted, palette='viridis')                   │
│  plt.title('Electricity Consumption by Indian States (2020-2021)', fontsize=16)                                 │
│  plt.xlabel('State', fontsize=12)                                                                               │
│  plt.ylabel('Consumption (TWh)', fontsize=12)                                                                   │
│  plt.xticks(rotation=45)                                                                                        │
│  plt.tight_layout()                                                                                             │
│  plt.legend(title='Year')                                                                                       │
│  plt.show()                                                                                                     │
│  ```                                                                                                            │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Texas, California, and Florida were the top three electricity-consuming U.S. states in 2023, primarily driven  │
│  by their large populations, industrial demands (e.g., manufacturing, technology sectors), and climate-related  │
│  energy use (e.g., cooling in warm climates). Texas, in particular, led due to its energy-intensive industries  │
│  and sprawling infrastructure, while California’s high consumption stemmed from its dense urbanization and      │
│  tech-driven economy. These states collectively accounted for over 30% of total U.S. electricity use,           │
│  underscoring the link between geographic scale, economic activity, and energy consumption patterns.            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Task: Combine outputs from analyzer, visualizer, and summarizer into a single report.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The top three electricity-consuming states in India are **Maharashtra**, **Tamil Nadu**, and **Karnataka**,    │
│  driven by their significant industrial activity, dense populations, and high urbanization. Maharashtra leads   │
│  due to its energy-intensive industries and sprawling infrastructure, while Tamil Nadu and Karnataka follow     │
│  due to their manufacturing sectors and demand for IT-tech infrastructure. A bar chart generated from           │
│  2020-2021 data (showing Maharashtra’s consumption rising from 120 to 130 TWh, Tamil Nadu from 90 to 95 TWh,    │
│  and Karnataka from 70 to 75 TWh) highlights consistent growth in these regions, underscoring their             │
│  disproportionate share of national electricity demand. Collectively, these states reflect a pattern where      │
│  economic scale and industrialization correlate with energy use, similar to the U.S. case, though at lower      │
│  absolute magnitudes.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The top three electricity-consuming states in India are **Maharashtra**, **Tamil Nadu**, and **Karnataka**, driven by their significant industrial activity, dense populations, and high urbanization. Maharashtra leads due to its energy-intensive industries and sprawling infrastructure, while Tamil Nadu and Karnataka follow due to their manufacturing sectors and demand for IT-tech infrastructure. A bar chart generated from 2020-2021 data (showing Maharashtra’s consumption rising from 120 to 130 TWh, Tamil Nadu from 90 to 95 TWh, and Karnataka from 70 to 75 TWh) highlights consistent growth in these regions, underscoring their disproportionate share of national electricity demand. Collectively, these states reflect a pattern where economic scale and industrialization correlate with energy use, similar to the U.S. case, though at lower absolute magnitudes.
